In [1]:
import cnvpytor
import subprocess

print(cnvpytor.__version__)

1.3.1


In [2]:
filename = "test_with_del__4_formated.fa"
command = f"cat {filename} | wc -l"
result = subprocess.check_output(command, shell=True)
line_count = int(result.strip())
line_count

162907

In [3]:
read_count = line_count * 50 * 30 / 202
read_count = round(read_count)
read_count

1209705

In [4]:
command = f"wgsim -d 300 -N {read_count} -1 101 -2 101 {filename} frag1Copy_test.fastq frag2Copy_test.fastq"
result = subprocess.check_output(command, shell=True)
print(result)

[wgsim] seed = 1706520855
[wgsim_core] calculating the total length of the reference sequence...
[wgsim_core] 1 sequences, total length: 8145284


b'chr22\t220\t-\tTA\t+\nchr22\t628\tA\tW\t+\nchr22\t1175\tT\tA\t-\nchr22\t1314\tG\tS\t+\nchr22\t2306\tG\tS\t+\nchr22\t2719\tT\tA\t-\nchr22\t3155\tC\tA\t-\nchr22\t4809\tG\tR\t+\nchr22\t5705\tC\tY\t+\nchr22\t5941\tG\tK\t+\nchr22\t6343\tC\tS\t+\nchr22\t6368\tC\tM\t+\nchr22\t6618\tG\tR\t+\nchr22\t8747\tG\tT\t-\nchr22\t9666\tT\tW\t+\nchr22\t10294\tT\tG\t-\nchr22\t10305\tA\tG\t-\nchr22\t10783\tA\tT\t-\nchr22\t10867\tG\tR\t+\nchr22\t13395\tC\tM\t+\nchr22\t13942\tT\tC\t-\nchr22\t14207\tA\tG\t-\nchr22\t14767\tT\tK\t+\nchr22\t17257\t-\tT\t+\nchr22\t17772\tA\tC\t-\nchr22\t18027\tT\tW\t+\nchr22\t19732\tC\tY\t+\nchr22\t20521\tT\tK\t+\nchr22\t20526\t-\tG\t+\nchr22\t21326\t-\tA\t-\nchr22\t21980\tA\t-\t-\nchr22\t22585\tC\t-\t-\nchr22\t27301\t-\tAG\t+\nchr22\t29149\tA\tR\t+\nchr22\t31136\tT\tW\t+\nchr22\t31568\tG\tA\t-\nchr22\t32029\tA\tG\t-\nchr22\t33412\tA\tM\t+\nchr22\t33773\tG\t-\t-\nchr22\t33913\tT\tC\t-\nchr22\t35549\tC\tM\t+\nchr22\t36819\tC\tA\t-\nchr22\t36921\tC\tT\t-\nchr22\t38346\tG\tS\t+\nc

In [5]:
# bowtie related stuff
!bowtie2-build reference.fa bias
!bowtie2 -p 4 -k 15 --no-mixed -x bias -1 frag1Copy_test.fastq -2 frag2Copy_test.fastq -S biasOut.sam

Settings:
  Output files: "bias.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  reference.fa
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:01
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:01
bmax according to bmaxDivN setting: 12826141
Using parameters --bmax 9619606 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 9619606 --dcv 1024
Constructing suffix-array element

In [6]:
# sam to bam file conversion
!samtools sort biasOut.sam -o biasOut_sorted.bam
!samtools index biasOut_sorted.bam

[bam_sort_core] merging from 1 files and 1 in-memory blocks...


In [7]:
app = cnvpytor.Root("test.pytor", create=True, max_cores=8)

In [8]:
app.rd(["biasOut_sorted.bam"])

In [9]:
app.calculate_histograms([100, 1000, 10000, 100000, 1000000, 10000000])

In [10]:
app.partition([100, 1000, 10000, 100000, 1000000, 10000000])

In [11]:
calls = app.call([100, 1000, 10000, 100000, 1000000, 10000000], print_calls=True)

duplication	chr22:643201-644800	1600	1.4859	7.244477e-06	9.232097e-58	1.000000e+00	1.000000e+00	0.0000	1.0000	0
duplication	chr22:1306101-1325100	19000	1.3975	0.000000e+00	5.623931e-266	0.000000e+00	4.641633e-237	0.0000	1.0000	0
duplication	chr22:2926201-2963100	36900	3.1273	0.000000e+00	1.018075e-65	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:4546401-4713300	166900	1.6662	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:6655001-6661800	6800	2.4292	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:6848301-6854500	6200	1.6768	0.000000e+00	4.231866e-239	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:8039401-8047800	8400	1.9513	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:9424001-9461600	37600	1.4867	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:13230601-13395900	165300	1.4076	0.000000e+00	0.000000e+00	0.0000

In [12]:
for bin_size in calls:
    print("Bin size: ", bin_size)

Bin size:  100
Bin size:  1000
Bin size:  10000
Bin size:  100000
Bin size:  1000000
Bin size:  10000000


In [13]:
print(f"{filename=}, Histogram Size: 100")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__4_formated.fa', Histogram Size: 100
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,643201,644800,1600,1.4859221351693344,0.0
duplication,chr22,1306101,1325100,19000,1.3975463541743007,0.0
duplication,chr22,2926201,2963100,36900,3.1272827937903735,0.0
duplication,chr22,4546401,4713300,166900,1.6661941733132197,0.0
duplication,chr22,6655001,6661800,6800,2.429179611621365,0.0
duplication,chr22,6848301,6854500,6200,1.676787693555141,0.0
duplication,chr22,8039401,8047800,8400,1.9513425405625093,0.0
duplication,chr22,9424001,9461600,37600,1.486681404880667,0.0
duplication,chr22,13230601,13395900,165300,1.4076226764442246,0.0
duplication,chr22,13449001,13453700,4700,1.3242289967466527,0.0
duplication,chr22,13478601,13512800,34200,4.025289225137081,0.0
duplication,chr22,15242601,15246400,3800,3.01835376290036,0.0
duplication,chr22,18662501,18664600,2100,1.45396786956914,0.0
duplication,chr22,18664801,18665900,1100,1.4945980365695595,0.0
duplication,

In [14]:
print(f"{filename=}, Histogram Size: 1000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__4_formated.fa', Histogram Size: 1000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,1306001,1325000,19000,1.3976755643074048,0.0
duplication,chr22,2926001,2963000,37000,3.1242192443951993,0.0
duplication,chr22,4546001,4713000,167000,1.6662848054735244,0.0
duplication,chr22,6655001,6662000,7000,2.3949879200335737,0.0
duplication,chr22,8040001,8048000,8000,1.949261535808257,0.0
duplication,chr22,9424001,9462000,38000,1.4832818558705052,0.0
duplication,chr22,13231001,13431000,200000,1.3450596607743812,0.0
duplication,chr22,13481001,13513000,32000,4.228470444748949,0.0
duplication,chr22,18705001,18881000,176000,2.7326664406484875,0.00014096247401351243
duplication,chr22,19259001,19268000,9000,3.247078825330423,0.0
duplication,chr22,21248001,21381000,133000,1.4783421089312099,0.00012744383510360785
duplication,chr22,21533001,21548000,15000,1.2960330061451328,0.0026642496154691275
duplication,chr22,21582001,21624000,42000,1.5575157024678148,0.0001

In [15]:
print(f"{filename=}, Histogram Size: 10,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__4_formated.fa', Histogram Size: 10,000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,4550001,4710000,160000,1.6659787798861438,0.0
duplication,chr22,6600001,7010000,410000,1.0935069436911136,0.0
duplication,chr22,7950001,8040000,90000,1.0668901400359179,0.0
duplication,chr22,13230001,13410000,180000,1.378876262208406,0.0
duplication,chr22,18710001,18880000,170000,2.721455020739251,0.00014548353802217994
duplication,chr22,21250001,21380000,130000,1.4555162885351232,0.00012751330072248494
duplication,chr22,25150001,25310000,160000,1.2677580248408404,0.00012919036787935968
duplication,chr22,28470001,28600000,130000,3.049457757885827,4.27468874060775e-05
duplication,chr22,30800001,30980000,180000,2.8474102977172278,2.5984733968793333e-05
duplication,chr22,31150001,31360000,210000,1.4088106633791777,4.966079900678402e-05
duplication,chr22,31870001,32250000,380000,1.2665093853310032,8.738459564437566e-05
duplication,chr22,35940001,36200000,260000

In [16]:
print(f"{filename=}, Histogram Size: 100,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__4_formated.fa', Histogram Size: 100,000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,3000001,6700000,3700000,1.077442820671135,0.0
duplication,chr22,45700001,46900000,1200000,1.5398458463636584,5.0283522243404934e-05
duplication,chr22,47100001,50100000,3000000,1.7510310396354947,5.109031790785054e-05


In [17]:
print(f"{filename=}, Histogram Size: 1,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__4_formated.fa', Histogram Size: 1,000,000
type,chromosome_name,start,end,size,cnv_level,q0


In [18]:
print(f"{filename=}, Histogram Size: 10,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__4_formated.fa', Histogram Size: 10,000,000
type,chromosome_name,start,end,size,cnv_level,q0


In [19]:
del app